# Notebook 1: Data Cleaning

This is the first notebook, responsible for
- Data Loading
- Data Cleaning
- Filtering only the required jobs descriptions by Job titles
- Stop words removal
- Tokenization
- BIO Tagging
- Extracting Skills from job description using Pattern Matching

# Imports

In [1]:
import pandas as pd
from google.colab import drive
from bs4 import BeautifulSoup
import ast
import spacy
from spacy.matcher import PhraseMatcher
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from tqdm import tqdm
tqdm.pandas(desc="Progress")

drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


# Data Loading

In [15]:
df1=pd.read_parquet("drive/Shareddrives/TM_project/jobpostings.parquet")
df2=pd.read_excel("drive/Shareddrives/TM_project/Technology Skills.xlsx")
df3=pd.read_excel("drive/Shareddrives/TM_project/Skills.xlsx")

In [16]:
df1.columns

Index(['Job Id', 'Job Title', 'Job Description', 'Company Name', 'Skills',
       'Qualification', 'Website Url'],
      dtype='object')

In [17]:
df1.head(10)

,Job Id,Job Title,Job Description,Company Name,Skills,Qualification,Website Url
0,89c41c519c3c491929e3082f0ee1d557,"Editor, Celebrations","<br/><br/>Gannett Co., Inc. (NYSE: GCI) is a s...",Gannett,"[Local Media, Editing, Journalism]","[Bachelor of Journalism (B.J.), Master of Jour...",https://www.gannett.com
1,ac0c91f394fa77a00ad72ee3440cb4b7,Software Engineer II,Overview </b> <br/><br/>Reporting to the...,ERT,"[Java, Application Architecture, CI, Data Stru...",[Bachelor of Computer Science (B.C.S.)],http://www.ert.com
2,4b5748411c4496f56ef33645a27840e0,Principal Software Architect,<br/> <br/>Digital technology has forever chan...,"Sovos Compliance, LLC.","[Java, CSS, Government Compliance, Global Comp...",None,https://sovos.com
3,7a7dac1bc98365216833008c0fbd063d,Strategy Program Manager,<br/> <br/> <b>Build your future with Sovos</b...,"Sovos Compliance, LLC.","[Government Compliance, Global Compliance, Com...",None,https://sovos.com
4,304aff90fd39fead183f48206f4070c7,Senior Contract Manager,Description<br/> <br/>LaBella Associates was e...,LaBella Associates,"[Legal, Disciplinary]","[Bachelor of Engineering (B.E./B.Eng.), Any Ba...",http://www.labellapc.com
5,f8e7c2fe7bfda8455ca2e18eb1beea5a,Newborn Hearing Screener,Overview </b> <br/><br/><b>Hearing Scree...,Oticon USA,"[Patient Care, vaccine, Nursing, Auditory]",[High School Diploma],http://www.Oticon.com
6,5c90575d962b567019643d4a8cea127f,Newborn Hearing Screener,Overview </b> <br/><br/><b>Hearing Scree...,Oticon USA,"[Patient Care, vaccine, Nursing, Auditory]",[High School Diploma],http://www.Oticon.com
7,d6583b8a3a0d7e72e884df39eda98576,Program Manager-Male Only,Overview </b> <br/><br/>The Program Mana...,Advocates,"[Budgetary, Consultation, Budget Development]",None,http://www.Advocates.org
8,5c89e0132ec782188f22757c68c315a1,"Senior Director, New Ventures","Senior Director, New Ventures<br/> ...",University of Massachusetts Medical School,"[Business Strategy, IP management, c, Investme...",None,https://umassmed.edu
9,54d9be0a01d5c48a47116f5d527154fe,Research Associate,<br/><b>Research Associate</b><br/> <br/>Job L...,Forrester Research,[Advisor],None,http://www.forrester.com


In [18]:
all_jobs = df1["Job Title"].value_counts()
all_jobs_50 = all_jobs[all_jobs>30].index

In [19]:
len(all_jobs)

86968

# Data Sampling

In [20]:
final_jobs=[
 'Software Engineer',
 'Senior Software Engineer',
 'Software Development Engineer',
 'Research Associate',
 'Financial Analyst',
 'Product Manager',
 'Business Analyst',
 'Business Development Representative',
 'Principal Software Engineer',
 'Data Engineer',
 'Data Scientist',
 'Mechanical Engineer',
 'Data Analyst',
 'Software Development Manager',
 'Senior Data Engineer',
 'Systems Engineer',
 'Associate Scientist',
 'Senior Product Manager',
 'Research Scientist',
 'Research Associate I',
 'DevOps Engineer',
 'Scientist',
 'Java Developer',
 'Software Engineer II',
 'Project Engineer',
 'Business Systems Analyst',
 'Program Director',
 'UX Designer',
 'Senior Program Manager',
 'Senior Software Development Engineer',
 'Engineer',
 'Software Developer',
 'Senior Business Analyst',
 'Senior Data Scientist'
]

In [21]:
len(final_jobs)

34

In [22]:
df_filtered = df1[df1['Job Title'].isin(final_jobs)]

len(df_filtered)

2288

# Data Cleaning

In [23]:
df_filtered[df_filtered["Job Description"].isnull()]

,Job Id,Job Title,Job Description,Company Name,Skills,Qualification,Website Url
113987,745c1de8aa6f17d56f9d7d2f845295d2,Financial Analyst,None,Optima Bank & Trust,"[Wealth Management, Accounting, Budgeting]","[Bachelor of Finance, Bachelor of Accounting]",http://www.optimabank.com
113991,6e707ea06537b3e4aa4d08ecfe64ef4e,Product Manager,None,Facebook,"[Product Development, Product Requirements, Pr...",None,http://www.facebook.com
113992,e2b53279e438915f1802305a552cda9a,Senior Software Engineer,None,Constant Contact,"[Java, CSS, UX, Unit Testing, Rex, CI, HTML, S...",None,http://www.constantcontact.com/index.jsp
122594,a8b413e57da4cc26b1c38842964c7b6c,Data Analyst,None,Drift: Conversational Marketing,"[Data Engineering, Scalability, Scala, Macro, ...",None,https://www.drift.com/
183685,21ca1e9bf16a7f12cf106f6595ad847c,Senior Software Engineer,None,Constant Contact,"[Java, CSS, UX, Unit Testing, Rex, CI, HTML, S...",None,http://www.constantcontact.com/index.jsp


In [24]:
no_null_df=df_filtered[df_filtered["Job Description"].notnull()]

In [25]:
col=["Job Id", "Job Description","Skills"]
df1_filtered = no_null_df[col]

In [26]:
def parse_html_tags(job_description): #function to remove HTML tags
    soup = BeautifulSoup(str(job_description), 'html.parser')
    parsed_text = soup.get_text()
    return parsed_text

In [27]:
df1_filtered.loc[:, 'Job Description'] = df1_filtered['Job Description'].astype(str)
df1_filtered.loc[:, 'Job Description'] = df1_filtered['Job Description'].progress_apply(parse_html_tags)

<ipython-input-27-f5ed6fc09aab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered.loc[:, 'Job Description'] = df1_filtered['Job Description'].astype(str)
Progress: 100%|██████████| 2283/2283 [00:05<00:00, 400.13it/s]
<ipython-input-27-f5ed6fc09aab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered.loc[:, 'Job Description'] = df1_filtered['Job Description'].progress_apply(parse_html_tags)


In [28]:
df1_filtered['Skills'].iloc[0]

'[Java, Application Architecture, CI, Data Structures, Kafka, JSON, SQL, Data Integration, Software Development Life Cycle, Application Development, Software Development, DevOps, Databases, XML, Database, Data Analysis, API, Oracle, AWS, Framework, Python, RESTful]'

In [29]:
def parse_skills_corrected(skills_str):
    if isinstance(skills_str, str):
        skills_list = skills_str.strip('[]').split(', ')
        skills_list = [skill.strip(" '") for skill in skills_list]
        return skills_list
    else:
        # If it's not a string (e.g., NaN), return an empty list or whatever makes sense in your context
        return []

# Apply the function to the entire 'Skills' column
df1_filtered['Skills'] = df1_filtered['Skills'].apply(parse_skills_corrected)

# Show the DataFrame to verify the operation
df1_filtered

<ipython-input-29-4da8717f1eac>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered['Skills'] = df1_filtered['Skills'].apply(parse_skills_corrected)


,Job Id,Job Description,Skills
1,ac0c91f394fa77a00ad72ee3440cb4b7,Overview Reporting to the Director of Data In...,"[Java, Application Architecture, CI, Data Stru..."
9,54d9be0a01d5c48a47116f5d527154fe,Research Associate Job LocationsUS-MA-Cambridg...,[Advisor]
232,2092de5cc1d91311a39b1cae0fee9c09,Schneider Electric creates connected technolog...,[]
333,46cbd86bab6441a53dee72855dc66ac6,"Working Location:MASSACHUSETTS, WESTBOROUGH;...","[Data Engineering, Schema Design, Informatica,..."
422,3502aa5ac916ad02f655f0b8525e9013,job summary: The Product Management Professio...,"[Product Requirements, Product Management, Pro..."
...,...,...,...
188578,90f107385cb990ffb9a3535a73bdd046,Auto req ID 55553BR Job Code I0857P IT Busines...,"[Information Systems, HTTP, BI Publisher, SQL,..."
188584,39aa8aa59412afe6753354be5868c1aa,Auto req ID 55453BR Job Code I0758P Applicatio...,"[Unix, Query Optimization, Java, Methodologies..."
188619,9975bf695f0fe6204cbd6a726228bdd9,Auto req ID 54782BR Job Code F0957P Financial ...,"[Financial Analysis, Accounting, Budget Analys..."
188624,f8fd3beb5dc778800272cf58aa9e8bb4,Auto req ID 54656BR Job Code I0958P IT Infrast...,"[HTTP, ITIL, Software Development, HTTPS, Ansi..."


In [30]:
job_ids=df1_filtered["Job Id"]

In [31]:
unique_job_ids = len(set(df1_filtered["Job Id"]))
unique_job_ids #All job IDs are unique

2283

In [32]:
df1_filtered = df1_filtered.reset_index(drop=True)
df1_filtered.index += 1
df1_filtered['Job Id'] = df1_filtered.index
df1_filtered

,Job Id,Job Description,Skills
1,1,Overview Reporting to the Director of Data In...,"[Java, Application Architecture, CI, Data Stru..."
2,2,Research Associate Job LocationsUS-MA-Cambridg...,[Advisor]
3,3,Schneider Electric creates connected technolog...,[]
4,4,"Working Location:MASSACHUSETTS, WESTBOROUGH;...","[Data Engineering, Schema Design, Informatica,..."
5,5,job summary: The Product Management Professio...,"[Product Requirements, Product Management, Pro..."
...,...,...,...
2279,2279,Auto req ID 55553BR Job Code I0857P IT Busines...,"[Information Systems, HTTP, BI Publisher, SQL,..."
2280,2280,Auto req ID 55453BR Job Code I0758P Applicatio...,"[Unix, Query Optimization, Java, Methodologies..."
2281,2281,Auto req ID 54782BR Job Code F0957P Financial ...,"[Financial Analysis, Accounting, Budget Analys..."
2282,2282,Auto req ID 54656BR Job Code I0958P IT Infrast...,"[HTTP, ITIL, Software Development, HTTPS, Ansi..."


In [33]:
job_id_simplified=df1_filtered['Job Id']

In [34]:
# Zipping the two lists together to create a new dataframe
zipped_df = pd.DataFrame(list(zip(job_ids, job_id_simplified)), columns=['Job Id', 'Job Id Simplified'])
zipped_df
merged_df_with_df_filtered = zipped_df.merge(df_filtered, on="Job Id", how="left")
merged_df_subset = merged_df_with_df_filtered[['Job Id', 'Job Id Simplified', 'Job Title']]
merged_df_subset.head()

,Job Id,Job Id Simplified,Job Title
0,ac0c91f394fa77a00ad72ee3440cb4b7,1,Software Engineer II
1,54d9be0a01d5c48a47116f5d527154fe,2,Research Associate
2,2092de5cc1d91311a39b1cae0fee9c09,3,Project Engineer
3,46cbd86bab6441a53dee72855dc66ac6,4,Data Engineer
4,3502aa5ac916ad02f655f0b8525e9013,5,Product Manager


# Creating Skill Taxonomy

In [ ]:
list1 = list(set(sum(df1_filtered['Skills'].tolist(), [])))
len(list1)

2022

In [ ]:
list2=list(set(df2["Example"]))
len(list2)

8869

In [ ]:
list3=list(df3["Element Name"].unique())
list3

['Reading Comprehension',
 'Active Listening',
 'Writing',
 'Speaking',
 'Mathematics',
 'Science',
 'Critical Thinking',
 'Active Learning',
 'Learning Strategies',
 'Monitoring',
 'Social Perceptiveness',
 'Coordination',
 'Persuasion',
 'Negotiation',
 'Instructing',
 'Service Orientation',
 'Complex Problem Solving',
 'Operations Analysis',
 'Technology Design',
 'Equipment Selection',
 'Installation',
 'Programming',
 'Operations Monitoring',
 'Operation and Control',
 'Equipment Maintenance',
 'Troubleshooting',
 'Repairing',
 'Quality Control Analysis',
 'Judgment and Decision Making',
 'Systems Analysis',
 'Systems Evaluation',
 'Time Management',
 'Management of Financial Resources',
 'Management of Material Resources',
 'Management of Personnel Resources']

In [ ]:
complete_list=list1+list2+list3
lowercase_list = [string.lower() for string in complete_list]

In [ ]:
full_list=list(set(lowercase_list))
len(lowercase_list),len(full_list)

(10926, 10758)

In [ ]:
full_list.remove("auto")

In [ ]:
fl=pd.Series(full_list)
fl.to_csv("skill_list.csv")

In [ ]:
set(full_list)

{'distance learning software',
 'shilstone seemix',
 'lji technologies lumberjack',
 'xfig',
 'call accounting software',
 'congruity technologies inspector',
 'microsoft visual sourcesafe',
 'server hardware',
 'pyxis medstation software',
 'eeo made simple aapmaker',
 'forum one communications projectspaces',
 'functional modeling software',
 'refworks',
 'airsmith flightprompt',
 'algorithmic software',
 'medmath',
 'aquifer test software',
 'power plants',
 'second foundation navimeat',
 'blackbaud luminate crm',
 'adp hr/benefits solution',
 'sparta systems trackwise',
 'spring batch',
 'honeywell wintress pacnet',
 'meteorjs',
 'oracle database',
 'gaussian gaussview',
 'inspection selection system iss',
 'propertythree',
 'hydrocad software solutions hydrocad',
 'work record software',
 'adobe systems adobe postscript',
 'industrial production manager and stock control software',
 'enterprise application integration eai software',
 'inphase technologies group inphase concept',
 

# Stop word removal and Tokenization

In [ ]:
df1_filtered['Job Description'] = df1_filtered['Job Description'].str.lower()

# Load stopwords
stop_words = set(stopwords.words('english'))

# Define a function to tokenize and remove stopwords
def tokenize_and_remove_stopwords(description):
    sentences = sent_tokenize(description)
    return [[word for word in word_tokenize(sentence) if word not in stop_words] for sentence in sentences]

# Use 'apply' instead of a loop for better performance
tokenized_descriptions = df1_filtered["Job Description"].progress_apply(tokenize_and_remove_stopwords)

Progress: 100%|██████████| 2283/2283 [00:17<00:00, 133.79it/s]


In [ ]:
df1_filtered["Tokenized Description"]=tokenized_descriptions
df1_filtered

,Job Id,Job Description,Skills,Tokenized Description
1,1,overview reporting to the director of data in...,"[Java, Application Architecture, CI, Data Stru...","[[overview, reporting, director, data, integra..."
2,2,research associate job locationsus-ma-cambridg...,[Advisor],"[[research, associate, job, locationsus-ma-cam..."
3,3,schneider electric creates connected technolog...,[],"[[schneider, electric, creates, connected, tec..."
4,4,"working location:massachusetts, westborough;...","[Data Engineering, Schema Design, Informatica,...","[[working, location, :, massachusetts, ,, west..."
5,5,job summary: the product management professio...,"[Product Requirements, Product Management, Pro...","[[job, summary, :, product, management, profes..."
...,...,...,...,...
2279,2279,auto req id 55553br job code i0857p it busines...,"[Information Systems, HTTP, BI Publisher, SQL,...","[[auto, req, id, 55553br, job, code, i0857p, b..."
2280,2280,auto req id 55453br job code i0758p applicatio...,"[Unix, Query Optimization, Java, Methodologies...","[[auto, req, id, 55453br, job, code, i0758p, a..."
2281,2281,auto req id 54782br job code f0957p financial ...,"[Financial Analysis, Accounting, Budget Analys...","[[auto, req, id, 54782br, job, code, f0957p, f..."
2282,2282,auto req id 54656br job code i0958p it infrast...,"[HTTP, ITIL, Software Development, HTTPS, Ansi...","[[auto, req, id, 54656br, job, code, i0958p, i..."


# BIO Tagging

In [ ]:
def get_bio_tags(lowercase_list, text_series):

    nlp = spacy.load("en_core_web_sm")
    matcher = PhraseMatcher(nlp.vocab)

    patterns = [nlp.make_doc(skill) for skill in lowercase_list]
    matcher.add("SKILL", None, *patterns)

    # Initialize an empty list to hold the BIO tags for each job description
    bio_tags_list = []
    for job_desc in tqdm(text_series):
        bio_tags_for_job_desc = []
        for sentence in job_desc:
            doc = spacy.tokens.Doc(nlp.vocab, words=sentence)
            bio_tags = ['O'] * len(doc)
            matches = matcher(doc)
            for match_id, start, end in matches:
                # Skip this match if it overlaps with a previous one
                if start > 0 and bio_tags[start - 1] in ('B', 'I'):
                    continue

                # The first token in the match gets a 'B' tag, the others get an 'I' tag
                bio_tags[start] = 'B'
                for i in range(start+1, end):
                    bio_tags[i] = 'I'
            bio_tags_for_job_desc.append(bio_tags)
        bio_tags_list.append(bio_tags_for_job_desc)

    return bio_tags_list

In [ ]:
bt = get_bio_tags(full_list, df1_filtered["Tokenized Description"])
df1_filtered['tags']=bt
df1_filtered

100%|██████████| 2283/2283 [00:04<00:00, 527.65it/s]


,Job Id,Job Description,Skills,Tokenized Description,tags
1,1,overview reporting to the director of data in...,"[Java, Application Architecture, CI, Data Stru...","[[overview, reporting, director, data, integra...","[[O, O, O, B, I, O, O, O, O, O, O, O, O, O, O,..."
2,2,research associate job locationsus-ma-cambridg...,[Advisor],"[[research, associate, job, locationsus-ma-cam...","[[O, O, O, O, O, O, O, O, O], [O, O, O, O, O, ..."
3,3,schneider electric creates connected technolog...,[],"[[schneider, electric, creates, connected, tec...","[[O, O, O, O, O, O, O, O, O, O, O, O, O], [O, ..."
4,4,"working location:massachusetts, westborough;...","[Data Engineering, Schema Design, Informatica,...","[[working, location, :, massachusetts, ,, west...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
5,5,job summary: the product management professio...,"[Product Requirements, Product Management, Pro...","[[job, summary, :, product, management, profes...","[[O, O, O, B, I, O, O, O, O, O, B, O, O, O, O,..."
...,...,...,...,...,...
2279,2279,auto req id 55553br job code i0857p it busines...,"[Information Systems, HTTP, BI Publisher, SQL,...","[[auto, req, id, 55553br, job, code, i0857p, b...","[[O, O, O, O, O, O, O, B, I, O, O, O, O, O, O,..."
2280,2280,auto req id 55453br job code i0758p applicatio...,"[Unix, Query Optimization, Java, Methodologies...","[[auto, req, id, 55453br, job, code, i0758p, a...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
2281,2281,auto req id 54782br job code f0957p financial ...,"[Financial Analysis, Accounting, Budget Analys...","[[auto, req, id, 54782br, job, code, f0957p, f...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
2282,2282,auto req id 54656br job code i0958p it infrast...,"[HTTP, ITIL, Software Development, HTTPS, Ansi...","[[auto, req, id, 54656br, job, code, i0958p, i...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,..."


In [ ]:
def extract_skills(tags_series, text_series):

    all_skills_found = []

    for tags, text in tqdm(zip(tags_series, text_series)):
        skills_found = []
        skill_phrase = []
        for sentence_tags, sentence in zip(tags, text):
            for word, tag in zip(sentence, sentence_tags):
                if tag == 'B':
                    # if a skill_phrase has been built, add it to skills_found
                    if skill_phrase:
                        skills_found.append(' '.join(skill_phrase))
                        skill_phrase = []
                    skill_phrase.append(word)
                elif tag == 'I':
                    skill_phrase.append(word)
        if skill_phrase:
            skills_found.append(' '.join(skill_phrase))

        unique_skills_found = list(set(skills_found))

        all_skills_found.append(unique_skills_found)
    return all_skills_found

In [ ]:
skills = extract_skills(df1_filtered['tags'], df1_filtered["Tokenized Description"])
df1_filtered['found_skills']=skills
df1_filtered

2283it [00:00, 7023.45it/s]


,Job Id,Job Description,Skills,Tokenized Description,tags,found_skills
1,1,overview reporting to the director of data in...,"[Java, Application Architecture, CI, Data Stru...","[[overview, reporting, director, data, integra...","[[O, O, O, B, I, O, O, O, O, O, O, O, O, O, O,...","[application development, json, sql, data anal..."
2,2,research associate job locationsus-ma-cambridg...,[Advisor],"[[research, associate, job, locationsus-ma-cam...","[[O, O, O, O, O, O, O, O, O], [O, O, O, O, O, ...","[application development, facebook, cio, twitt..."
3,3,schneider electric creates connected technolog...,[],"[[schneider, electric, creates, connected, tec...","[[O, O, O, O, O, O, O, O, O, O, O, O, O], [O, ...","[software house, microsoft word, project manag..."
4,4,"working location:massachusetts, westborough;...","[Data Engineering, Schema Design, Informatica,...","[[working, location, :, massachusetts, ,, west...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[data science, data engineering, methodologies..."
5,5,job summary: the product management professio...,"[Product Requirements, Product Management, Pro...","[[job, summary, :, product, management, profes...","[[O, O, O, B, I, O, O, O, O, O, B, O, O, O, O,...","[product management, product requirements, des..."
...,...,...,...,...,...,...
2279,2279,auto req id 55553br job code i0857p it busines...,"[Information Systems, HTTP, BI Publisher, SQL,...","[[auto, req, id, 55553br, job, code, i0857p, b...","[[O, O, O, O, O, O, O, B, I, O, O, O, O, O, O,...","[critical thinking, sql, https, data analysis,..."
2280,2280,auto req id 55453br job code i0758p applicatio...,"[Unix, Query Optimization, Java, Methodologies...","[[auto, req, id, 55453br, job, code, i0758p, a...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[programming, debugging, data security, adviso..."
2281,2281,auto req id 54782br job code f0957p financial ...,"[Financial Analysis, Accounting, Budget Analys...","[[auto, req, id, 54782br, job, code, f0957p, f...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[budget management, oracle, project cost, micr..."
2282,2282,auto req id 54656br job code i0958p it infrast...,"[HTTP, ITIL, Software Development, HTTPS, Ansi...","[[auto, req, id, 54656br, job, code, i0958p, i...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[ca, https, drawing, critical thinking, servic..."


In [ ]:
def get_missing_skills(skills, found_skills):
    # Convert both lists to lowercase sets for comparison
    skills_set = set(skill.lower() for skill in skills if isinstance(skill, str))
    found_skills_set = set(skill.lower() for skill in found_skills if isinstance(skill, str))
    # Find skills that are in 'skills' but not in 'found_skills'
    missing_skills = skills_set - found_skills_set
    return list(missing_skills)

# Apply the function to each row to create the new column 'ISNIFS'
df1_filtered['ISNIFS'] = df1_filtered.apply(lambda row: get_missing_skills(row['Skills'], row['found_skills']), axis=1)
df1_filtered

,Job Id,Job Description,Skills,Tokenized Description,tags,found_skills,ISNIFS
1,1,overview reporting to the director of data in...,"[Java, Application Architecture, CI, Data Stru...","[[overview, reporting, director, data, integra...","[[O, O, O, B, I, O, O, O, O, O, O, O, O, O, O,...","[application development, json, sql, data anal...","[databases, software development, framework, ci]"
2,2,research associate job locationsus-ma-cambridg...,[Advisor],"[[research, associate, job, locationsus-ma-cam...","[[O, O, O, O, O, O, O, O, O], [O, O, O, O, O, ...","[application development, facebook, cio, twitt...",[]
3,3,schneider electric creates connected technolog...,[],"[[schneider, electric, creates, connected, tec...","[[O, O, O, O, O, O, O, O, O, O, O, O, O], [O, ...","[software house, microsoft word, project manag...",[]
4,4,"working location:massachusetts, westborough;...","[Data Engineering, Schema Design, Informatica,...","[[working, location, :, massachusetts, ,, west...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[data science, data engineering, methodologies...","[rdbms, scripting]"
5,5,job summary: the product management professio...,"[Product Requirements, Product Management, Pro...","[[job, summary, :, product, management, profes...","[[O, O, O, B, I, O, O, O, O, O, B, O, O, O, O,...","[product management, product requirements, des...",[]
...,...,...,...,...,...,...,...
2279,2279,auto req id 55553br job code i0857p it busines...,"[Information Systems, HTTP, BI Publisher, SQL,...","[[auto, req, id, 55553br, job, code, i0857p, b...","[[O, O, O, O, O, O, O, B, I, O, O, O, O, O, O,...","[critical thinking, sql, https, data analysis,...","[databases, data conversion, frameworks]"
2280,2280,auto req id 55453br job code i0758p applicatio...,"[Unix, Query Optimization, Java, Methodologies...","[[auto, req, id, 55453br, job, code, i0758p, a...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[programming, debugging, data security, adviso...","[oracle, agile, http, problem solving, corona,..."
2281,2281,auto req id 54782br job code f0957p financial ...,"[Financial Analysis, Accounting, Budget Analys...","[[auto, req, id, 54782br, job, code, f0957p, f...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[budget management, oracle, project cost, micr...",[]
2282,2282,auto req id 54656br job code i0958p it infrast...,"[HTTP, ITIL, Software Development, HTTPS, Ansi...","[[auto, req, id, 54656br, job, code, i0958p, i...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[ca, https, drawing, critical thinking, servic...","[http, aws, amazon web services (aws), corona,..."


In [ ]:
df_with_isnifs = df1_filtered[df1_filtered['ISNIFS'].apply(len) > 0]

len(df_with_isnifs[['Skills', 'found_skills', 'ISNIFS']])

1553

In [ ]:
unique_skills_isnifs = set() #805 skills in total
for skills in df_with_isnifs['ISNIFS']:
    unique_skills_isnifs.update(skills)
skills_not_in_full_list = unique_skills_isnifs - set(full_list)

skills_not_in_full_list

set()

model errors

Many skills in the skills column have not correctly been extracted by the pattern matching. letting them be for now. All the sets found in the column ISNIFS (in skills but not in found skills) are present in the full skill list but there are errors during pattern matching.

In [ ]:
df1_filtered

,Job Id,Job Description,Skills,Tokenized Description,tags,found_skills,ISNIFS
1,1,overview reporting to the director of data in...,"[Java, Application Architecture, CI, Data Stru...","[[overview, reporting, director, data, integra...","[[O, O, O, B, I, O, O, O, O, O, O, O, O, O, O,...","[application development, json, sql, data anal...","[databases, software development, framework, ci]"
2,2,research associate job locationsus-ma-cambridg...,[Advisor],"[[research, associate, job, locationsus-ma-cam...","[[O, O, O, O, O, O, O, O, O], [O, O, O, O, O, ...","[application development, facebook, cio, twitt...",[]
3,3,schneider electric creates connected technolog...,[],"[[schneider, electric, creates, connected, tec...","[[O, O, O, O, O, O, O, O, O, O, O, O, O], [O, ...","[software house, microsoft word, project manag...",[]
4,4,"working location:massachusetts, westborough;...","[Data Engineering, Schema Design, Informatica,...","[[working, location, :, massachusetts, ,, west...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[data science, data engineering, methodologies...","[rdbms, scripting]"
5,5,job summary: the product management professio...,"[Product Requirements, Product Management, Pro...","[[job, summary, :, product, management, profes...","[[O, O, O, B, I, O, O, O, O, O, B, O, O, O, O,...","[product management, product requirements, des...",[]
...,...,...,...,...,...,...,...
2279,2279,auto req id 55553br job code i0857p it busines...,"[Information Systems, HTTP, BI Publisher, SQL,...","[[auto, req, id, 55553br, job, code, i0857p, b...","[[O, O, O, O, O, O, O, B, I, O, O, O, O, O, O,...","[critical thinking, sql, https, data analysis,...","[databases, data conversion, frameworks]"
2280,2280,auto req id 55453br job code i0758p applicatio...,"[Unix, Query Optimization, Java, Methodologies...","[[auto, req, id, 55453br, job, code, i0758p, a...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[programming, debugging, data security, adviso...","[oracle, agile, http, problem solving, corona,..."
2281,2281,auto req id 54782br job code f0957p financial ...,"[Financial Analysis, Accounting, Budget Analys...","[[auto, req, id, 54782br, job, code, f0957p, f...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[budget management, oracle, project cost, micr...",[]
2282,2282,auto req id 54656br job code i0958p it infrast...,"[HTTP, ITIL, Software Development, HTTPS, Ansi...","[[auto, req, id, 54656br, job, code, i0958p, i...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[ca, https, drawing, critical thinking, servic...","[http, aws, amazon web services (aws), corona,..."


In [ ]:
cols_for_modelling = ['Job Id', 'Tokenized Description', 'tags']
df_transformed= df1_filtered[cols_for_modelling]

In [ ]:
df_transformed.to_parquet("transformed_data.parquet")

In [ ]:
model_results=pd.read_parquet("/content/drive/Shareddrives/TM_project/pred_results.parquet")

In [ ]:
# Merging df1_filtered with merged_df_subset
# 'Job Id' in df1_filtered corresponds to 'Job Id Simplified' in merged_df_subset
merged_final_df = df1_filtered.merge(merged_df_subset, left_on='Job Id', right_on='Job Id Simplified')
merged_final_df.to_parquet("merged_data.parquet")